### 想解釋各個國家粗出生率的變異，利用古典迴歸模型做出OLSE，並驗證解釋變數與反應變數關係符合直觀預期。
###### 以2017年的資料為例，資料來源為世界銀行。

In [1]:
import numpy as np
from numpy.lib.scimath import log
import pandas as pd

In [2]:
#data source : The World Bank (2017)
df = pd.read_csv("Regression Data.csv", encoding = "utf-8")

In [3]:
df

,Country Name,CrudeBirthRate,GNI_per_capita,dependencyRatioOld,expectancyLife,HDI,developmentLevel
0,Afghanistan,32.507,560,4.763322,64.047,0.498,0
1,Albania,11.745,4320,18.997890,78.495,0.785,0
2,Algeria,22.292,3940,9.628360,76.293,0.754,0
3,Angola,41.347,3570,4.734532,61.809,0.581,0
4,Argentina,16.975,13030,17.520505,76.738,0.825,0
...,...,...,...,...,...,...,...
170,Uzbekistan,22.100,2000,6.616025,71.420,0.710,0
171,Vanuatu,25.524,2920,7.358980,72.334,0.603,0
172,Vietnam,16.450,2160,10.244811,76.454,0.694,0
173,Zambia,37.833,1290,4.701909,62.255,0.588,0


In [4]:
#反應變數：粗出生率，為一175維的行向量 Y
Y = np.array(df["CrudeBirthRate"].tolist()).reshape((-1,1))

#截距項
interceptList = []
for i in range(len(df)):
    interceptList.append(1)
intercept = np.array(interceptList).reshape((-1,1))

#解釋變數 1：人均國民所得取自然對數
ln_GNI = np.array(log(df["GNI_per_capita"]).tolist()).reshape((-1,1))

#解釋變數 2：該國扶老比取自然對數
ln_DRO = np.array(log(df["dependencyRatioOld"]).tolist()).reshape((-1,1))

#解釋變數 3：該國平均壽命
EL = np.array(df["expectancyLife"].tolist()).reshape((-1,1))

#解釋變數 4：HDI是否大於0.9，大於0.9者稱發展程度為1，反之則為0。
DL = np.array(df["developmentLevel"].tolist()).reshape((-1,1))

#將截距項與解釋變數們合併成一個175 by 5的矩陣 X
X = np.hstack((intercept, ln_GNI, ln_DRO, EL, DL))

In [5]:
#OLSE
OLSE = np.linalg.inv(X.transpose()@X)@X.transpose()@Y
OLSE

array([[93.07598036],
       [-2.18817586],
       [-5.93035839],
       [-0.56225649],
       [ 6.58101126]])

In [8]:
print("CrudeBirthRate = {} + ({}) * ln_GNI + ({}) * ln_DRO + ({}) * EL + {} * DL".format(OLSE[0][0], OLSE[1][0], OLSE[2][0], OLSE[3][0], OLSE[4][0]))

CrudeBirthRate = 93.07598035770002 + (-2.188175858373376) * ln_GNI + (-5.930358393692592) * ln_DRO + (-0.5622564883747256) * EL + 6.581011260295781 * DL


### 由上面OLSE可知：
#### 1. 人均國民所得與粗出生率有反向關係：因為開發中國家傾向有高出生率，而像台灣或其他大多數已開發國家，出生率則較低。
#### 2. 扶老比與粗出生率有反向關係：因為當年輕人扶養負擔越重，越不想生小孩，亦符合直觀預期。
#### 3. 該國平均壽命與粗出生率有反向關係：因為高平均餘命的國家，代表有著健全的醫療體系，不會有想多生一點的動機。
#### 4. HDI>0.9與粗出生率有正向關係：此項係為捕捉像瑞典、芬蘭等高社會福利的國家，雖為已開發國家，但其社會福利制度健全鼓勵生育，故也符合預期。